In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from osgeo import gdal
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.model_selection import train_test_split

2025-03-01 21:25:20.344706: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 21:25:20.345228: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 21:25:20.347891: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-01 21:25:20.354899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740864320.365096  204324 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740864320.36

In [3]:
img = cv2.imread("/home/h00pyfr00d/Downloads/data/20230215-SE2B-CGG-GBR-MS3-L3-RGB-preview.jpg", cv2.IMREAD_COLOR)
lidar = gdal.Open("/home/h00pyfr00d/Downloads/data/DSM_TQ0075_P_12757_20230109_20230315.tif")
# Read the first band of the LIDAR data
lidar_band = lidar.GetRasterBand(1)
lidar_data = lidar_band.ReadAsArray()

/usr/lib/python3/dist-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [7]:
ground_threshold = 18
# Create a mask for areas above the ground threshold
above_ground_mask = lidar_data > ground_threshold
# Apply the mask to the LIDAR data
above_ground_lidar_data = np.where(above_ground_mask, lidar_data, np.nan)

nrg_img = cv2.imread("/home/h00pyfr00d/Downloads/data/20230215-SE2B-CGG-GBR-MS3-L3-NRG-preview.jpg", cv2.IMREAD_COLOR)

# Split the NRG image into its respective bands
nir_band = nrg_img[:, :, 0].astype(float)  # Near-Infrared band
red_band = nrg_img[:, :, 2].astype(float)  # Red band

# Subtract the bands
band_difference = nir_band - red_band


In [8]:
# Define a threshold for the band difference to identify greenery
greenery_threshold = -50

# Create a mask for areas that are not greenery
non_greenery_mask = band_difference > greenery_threshold

# Resize the mask to match the shape of above_ground_lidar_data
non_greenery_mask_resized = cv2.resize(non_greenery_mask.astype(np.uint8), (above_ground_lidar_data.shape[1], above_ground_lidar_data.shape[0]), interpolation=cv2.INTER_NEAREST).astype(bool)

# Apply the resized mask to the above ground LIDAR data
above_ground_lidar_data_no_greenery = np.where(non_greenery_mask_resized, above_ground_lidar_data, np.nan)